In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///database.db

In [ ]:
contributions_q = '''
SELECT
    ac.author_name,
    anc.n_commits,
    ac.n_lines_added,
    ac.n_lines_deleted,
    ac.n_lines_changed
FROM
    authors_contibutions ac JOIN authors_n_commits anc USING(author_name)
ORDER BY
    anc.n_commits DESC;
'''
contributions = %sql $contributions_q
contributions = contributions.DataFrame()

In [ ]:
fig = px.pie(contributions, title='n_commits per author', values='n_commits', names='author_name')
fig.show()

fig = go.Figure(data=[
    go.Bar(name='n_lines_added', x=contributions.author_name, y=contributions.n_lines_added, base=0),
    go.Bar(name='n_lines_deleted', x=contributions.author_name, y=contributions.n_lines_deleted, base=list(map(lambda x: -x, contributions.n_lines_deleted))),
])
fig.update_layout(title='n_lines_added and n_lines_deleted per author', barmode='stack')
fig.show()

In [ ]:
changes_over_time_q = '''
SELECT
    cc.*,
    AVG(cc.n_files) OVER (ORDER BY cc.commit_date ROWS BETWEEN 2 PRECEDING AND 2 FOLLOWING) AS avg_files,
    c.message AS commit_message
FROM
    commits_changes cc JOIN commits c ON cc.commit_hash = c.hash;
'''

changes_over_time = %sql $changes_over_time_q
changes_over_time = changes_over_time.DataFrame().astype({'commit_date': 'datetime64[ns]'})

In [ ]:
fig = make_subplots(specs=[[{'secondary_y': True}]])
fig.update_xaxes(title_text='date')

fig.add_trace(go.Scatter(x=changes_over_time.commit_date, y=changes_over_time.avg_files, name='n_files', line_shape='hvh', hovertext=changes_over_time.commit_message))
fig.update_yaxes(title_text='file_count')

fig.add_trace(go.Scatter(x=changes_over_time.commit_date, y=changes_over_time.avg_complexity, name='avg_complexity', line_shape='hvh', hovertext=changes_over_time.commit_message), secondary_y=True)
fig.update_yaxes(title_text='avg_complexity', secondary_y=True)

fig.show()

In [ ]:
coupling_q = 'SELECT * FROM files_coupling fc;'
coupling = %sql $coupling_q
coupling = coupling.DataFrame()

display(coupling)

# TODO: Chord diagram, ideally grouped by package

In [ ]:
changes_x_size_complexity_q = '''
SELECT
    f."path" AS file_path,
    COUNT(s.file_path) AS n_changes,
    AVG(s.code_lines) AS avg_code_lines,
    AVG(s.complexity) AS avg_complexity
FROM
    files f,
    snapshots s
WHERE
    s.file_path = f."path" AND
    s.changed = TRUE AND
    f.language in ('scala', 'java')
GROUP BY
    f.file_name;
'''

changes_x_size_complexity = %sql $changes_x_size_complexity_q
changes_x_size_complexity = changes_x_size_complexity.DataFrame()

In [ ]:
fig = px.scatter(
    changes_x_size_complexity,
    x='n_changes',
    y='file_path',
    size='avg_code_lines',
    color='avg_complexity',
    title='Scala and Java hotspots',
    hover_data=['file_path'],
    color_continuous_scale=["green", "red", "black"],
    height=800,
)
fig.update_layout(
    yaxis = dict(
        tickmode='array',
        tickson='boundaries',
        showline=True,
    )
)
fig.show()

# TODO: Circle Packing Chart by package